## Date Preprocess

In [132]:
import pandas as pd
import numpy as np

#L = 7 or 14
L = 7

# Pick out L-days data to training data
def Data_compare(data,window):
    view_count = 0
    request_num = 0
    start_time = -1
    timestamp = 0
    end_time = 0
    idx = []
    for i in range(data.shape[0]):
        timestamp = int(str(data.iloc[i,0])[3:10])
        if start_time == -1:
            start_time = timestamp + (window-1) * 86400
            end_time = start_time + window * 86400
        if end_time < timestamp:
            break
        if (timestamp >= start_time and timestamp <= end_time):
            idx.append(i)
    print(idx[0],'min',idx[-1],'max')
    return idx

data = pd.read_csv("./test_20180920 _orgin")
data["Video_ID"] = pd.Categorical(data['Video_ID'], categories=data["Video_ID"].unique()).codes
data = data[['Req_time','Video_ID','Video_Category','Like_Count','Dislike_Count','Comment_Count','Local_View','Video_Count']]
idx = Data_compare(data,L)

train_data = data.iloc[:idx[0]]
test_data = data.iloc[idx[0]:idx[-1],1]
test_data.to_csv("./testing_1001_video.csv",index=0)

print(train_data.shape)
print(test_data.shape)


110415 min 248457 max
(110415, 8)
(138042,)


## Remove "View Count = 0" Rows and Duplicate Row

In [133]:
train_data = train_data.drop_duplicates(subset=['Video_ID'], keep="first")
print(train_data.shape)
train_data = train_data[train_data.Video_Count != 0]
train_data["Video_ID"].to_csv("./training_1001_video.csv",index=0)

(57353, 8)


In [134]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer


onehot_encoding = pd.get_dummies(train_data['Video_Category'],prefix='Video_Category')
print(train_data.shape)
X_train = train_data.drop(['Video_Category'],axis=1)
print(X_train.shape)
X_train = pd.concat([onehot_encoding,X_train],axis=1)
print(X_train.shape)

transformer = FunctionTransformer(np.log1p)

# MinMax standardization
scaler = MinMaxScaler()
#scaler = StandardScaler()
x_train = scaler.fit_transform(X_train.drop(['Video_Count'],axis=1).as_matrix())
#y_train = scaler.fit_transform(X_train['Video_Count'].astype(int).as_matrix().reshape(X_train.shape[0],1))

y_train = transformer.transform(X_train['Video_Count'].astype(int).as_matrix().reshape(1,-1))
y_train = y_train.reshape(-1,1)
print(x_train[1])
print(x_train.shape)
print(y_train.shape)
#original_data = train_data.drop(original_data.columns[1],axis=1)
#original_data = np.delete(original_data,[0,-1],axis=1)




(57350, 8)
(57350, 7)
(57350, 24)
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.11890336e-05 1.74361836e-05 8.54023413e-05
 6.86487859e-05 2.38545694e-04 0.00000000e+00]
(57350, 23)
(57350, 1)


C:\Users\wmnet\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


## NN model

In [136]:
def neural_net_model(X_data,input_dim):
    W_1 = tf.Variable(tf.random_uniform([input_dim,100]))
    b_1 = tf.Variable(tf.zeros([100]))
    layer_1 = tf.add(tf.matmul(X_data,W_1), b_1)
    layer_1 = tf.nn.relu(layer_1)

    # layer 1 multiplying and adding bias then activation function
    W_2 = tf.Variable(tf.random_uniform([100,100]))
    b_2 = tf.Variable(tf.zeros([100]))
    layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
    layer_2 = tf.nn.relu(layer_2)
    # layer 2 multiplying and adding bias then activation function
    W_O = tf.Variable(tf.random_uniform([100,1]))
    b_O = tf.Variable(tf.zeros([1]))
    output = tf.add(tf.matmul(layer_2,W_O), b_O)
    # O/p layer multiplying and adding bias then activation function
    # notice output layer has one node only since performing #regression
    return output

"""
neural_net_model is function applying 2 hidden layer feed forward neural net.
Weights and biases are abberviated as W_1,W_2 and b_1, b_2 
These are variables with will be updated during training.
"""
#xs = tf.placeholder("float")
#ys = tf.placeholder("float")
batch_size = 64
global_step = tf.Variable(0, trainable=False)
display_step = 1000
c_t = []


xs = tf.placeholder(tf.float32, [None,23])
ys = tf.placeholder(tf.float32, [None, 1])

output = neural_net_model(xs,23)
cost = tf.reduce_mean(tf.square(output-ys))
# our mean squared error cost function
train = tf.train.GradientDescentOptimizer(1e-6).minimize(cost,global_step=global_step)
# Gradinent Descent optimiztion just discussed above for updating weights and biases


with tf.Session() as sess:
    # Initiate session and initialize all vaiables
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    #saver.restore(sess,'yahoo_dataset.ckpt')
    for i in range(100):
        permutation=np.random.permutation(x_train.shape[0])
        x_batch = x_train[permutation,:]
        y_batch = y_train[permutation]
        x_batch = np.array_split(x_batch, x_train.shape[0] // batch_size)
        y_batch = np.array_split(y_batch, x_train.shape[0] // batch_size)
        #print(x_batch[0].shape)
        #print(y_batch[0].shape)
        for batch_x,batch_y in zip(x_batch,y_batch):
            # Run optimization op (backprop)
            sess.run(train, feed_dict={xs: batch_x, ys: batch_y.reshape(batch_y.shape[0],1)})
            step = sess.run(global_step)
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss = sess.run(cost, feed_dict={xs: batch_x,ys: batch_y.reshape(batch_y.shape[0],1)})
                print("Step " + str(step) + ", Minibatch Loss= " +"{:.4f}".format(loss))
    
        #for j in range(x_train.shape[0]):
            #sess.run([cost,train],feed_dict={xs:x_train[j,:].reshape(1,23), ys:y_train[j]})
            # Run cost and train with each sample
        c_t.append(sess.run(cost, feed_dict={xs:x_train,ys:y_train.reshape(y_train.shape[0],1)}))
        #c_test.append(sess.run(cost, feed_dict={xs:X_test,ys:y_test}))
        print('Epoch :',i,'Cost :',c_t[i])
    pred = sess.run(output, feed_dict={xs:x_train})
    # predict output of test data after training
    #print('Cost :',sess.run(cost, feed_dict={xs:X_test,ys:y_test}))
    #y_test = denormalize(df_test,y_test)
    #pred = denormalize(df_test,pred)

Step 1, Minibatch Loss= 931601.3750
Epoch : 0 Cost : 27.649727
Step 1000, Minibatch Loss= 22.5020
Epoch : 1 Cost : 26.115046
Step 2000, Minibatch Loss= 26.6270
Epoch : 2 Cost : 24.711737
Step 3000, Minibatch Loss= 22.4074
Epoch : 3 Cost : 23.45364
Step 4000, Minibatch Loss= 21.9638
Epoch : 4 Cost : 22.298676
Step 5000, Minibatch Loss= 19.7384
Epoch : 5 Cost : 21.328468
Step 6000, Minibatch Loss= 18.2394
Epoch : 6 Cost : 20.244238
Step 7000, Minibatch Loss= 25.7811
Epoch : 7 Cost : 19.398735
Step 8000, Minibatch Loss= 19.7897
Epoch : 8 Cost : 18.48216
Epoch : 9 Cost : 17.72505
Step 9000, Minibatch Loss= 13.9345
Epoch : 10 Cost : 17.018612
Step 10000, Minibatch Loss= 13.7821
Epoch : 11 Cost : 16.27776
Step 11000, Minibatch Loss= 15.8305
Epoch : 12 Cost : 15.642202
Step 12000, Minibatch Loss= 12.6220
Epoch : 13 Cost : 15.061877
Step 13000, Minibatch Loss= 13.0732
Epoch : 14 Cost : 14.485539
Step 14000, Minibatch Loss= 11.3291
Epoch : 15 Cost : 13.979897
Step 15000, Minibatch Loss= 17.6708

## Save Predicted Data

In [137]:
np.savetxt("pred_2018_10_10.csv",pred, delimiter=",")


## Sort and Calculate Hit Rate

In [138]:
result = pd.read_csv("pred_2018_10_10.csv",header=None)
train_ID = pd.read_csv("training_1001_video.csv",header=None)
test_ID = pd.read_csv("testing_1001_video.csv",header=None)
final_result = pd.concat([result,train_ID],axis=1,ignore_index=True)
final_result = final_result.sort([0],ascending=False)
print(final_result)

               0      1
48     31.095615     48
20713  29.874971  20714
4687   25.445910   4687
2906   24.593761   2906
4688   23.266281   4688
53679  21.101404  53681
45824  19.797440  45826
4701   18.819902   4701
5357   18.592070   5357
1188   17.982145   1188
3982   17.923649   3982
11654  17.592016  11655
48455  17.366436  48457
3556   17.358101   3556
2989   17.212898   2989
58     17.019949     58
36950  16.775280  36951
14404  16.578680  14405
3577   16.228876   3577
3395   16.226210   3395
56296  16.196896  56298
21978  16.195562  21979
53803  16.161816  53805
7221   16.026272   7221
3703   15.857057   3703
8095   15.686289   8096
3053   15.600843   3053
2276   15.493347   2276
3003   15.449720   3003
9949   15.285483   9950
...          ...    ...
6217    7.799737   6217
2061    7.798448   2061
6052    7.797279   6052
4614    7.796119   4614
962     7.795404    962
5841    7.794765   5841
5933    7.794611   5933
4601    7.785686   4601
3285    7.785018   3285
4591    7.783528

C:\Users\wmnet\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [141]:
total_size = test_ID.shape[0]
# cache size range 500,5000,50000
for length in range(100,100+1,500):
    hit = 0
    cache_cotainer = final_result.iloc[:length+1,1]
    for item in test_ID.iloc[:,0]:
        if item in cache_cotainer:
            hit += 1
    print("Cach Size %5f ---Hit Rate: %f" % (length,hit/total_size*100))

Cach Size 100.000000 ---Hit Rate: 1.045334
